In [15]:
import glob
import pandas as pd
import datetime
import IPython.display as display

In [16]:
dfs = []
filenames = []
for filename in glob.glob('../selectedData/*'):
    print
    df = pd.read_csv(filename)
    df.name = filename.split('/')[-1].split('.')[0]
    df['Date'] =  pd.to_datetime(df['Date'])
    df.sort_index(axis=1, ascending=False)
    dfs.append(df)
    filenames.append(df.name)

In [5]:
for i in range(len(dfs)):
    df = dfs[i]
    dfs[i] = df[(df['Date']>='2016-11-07') & (df['Date']<='2017-11-10')].reset_index(drop=True)
    dfs[i].name = df.name

In [6]:
checkDate = []
for df in dfs:
    checkDate.append(df[(df['Date']>='2016-11-07') & (df['Date']<='2017-11-10')]['Date'].tolist() == dfs[0][(dfs[0]['Date']>='2016-11-07') & (dfs[0]['Date']<='2017-11-10')]['Date'].tolist())
dateChecked = True
for i in checkDate:
    if(not i):
        dateChecked = False
        print("Date does not match for some.")
        break
if dateChecked:
    print("Date checked successfully")


Date checked successfully


In [11]:
def preprocess(df):
    '''
    Values are calculated w.r.t to the previous week. 
    Delta for week 0 is calculated as open price for start of week 0 - week -1
    High for week 0: Highest price from start of week -1 to start of week 1
    Low for week 0: Lowest price from start of week -1 to start of week 0 
    '''
    currentDate = datetime.datetime(2016,11,17)
    endDate = datetime.datetime(2017,11,10)
    df_ = pd.DataFrame(columns=['Date','Open','Delta','FractionalIncrease','High','Low','Positive'])
    while currentDate <= endDate:
        i = []
        tempDate = currentDate
        while(len(i) == 0):
            i = df.index[df.Date == tempDate].values
            tempDate += datetime.timedelta(days=1)
        i = i[0]
        
        prevI = []
        tempDate = currentDate - datetime.timedelta(days=7)
        while(len(prevI) == 0):
            prevI = df.index[df.Date == tempDate].values
            tempDate += datetime.timedelta(days=1)
        prevI = prevI[0]
        
        date = df.iloc[i]['Date']
        open_ = df.iloc[i]['Open']
        delta = df.iloc[i]['Open'] - df.iloc[prevI]['Open']
        fractionalIncrease = delta/df.iloc[prevI]['Open']
        high = max(df.iloc[prevI:i]['High'])
        low = min(df.iloc[prevI:i]['Low'])
        positive = delta >= 0
        df_.loc[len(df_)] = [date,open_,delta,fractionalIncrease,high,low,positive]
        currentDate += datetime.timedelta(days=7)
    return df_
        


In [14]:
dfs

[]

In [12]:
i = 0
for df in dfs:
    output = preprocess(df)
    output.to_csv("processedSelected/{}.csv".format(filenames[i]), index=False)
    i += 1

In [9]:
!ls

bowSentiment.ipynb  dataAugment.ipynb  GoogleScrap.ipynb  Sentiment.ipynb


In [10]:
!pwd

/home/vincent/cds/project/stockPredict/dataprocessing
